In [2]:
import os
os.listdir()

['.ipynb_checkpoints',
 'analysis.ipynb',
 'orders.csv',
 'restaurants.sql',
 'users.json']

In [3]:
import pandas as pd
import json
import sqlite3


In [4]:
orders = pd.read_csv("orders.csv")

In [5]:
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [ ]:
orders.info()

In [6]:
with open("users.json", "r") as f:
    users_data = json.load(f)

In [7]:
users = pd.DataFrame(users_data)

In [8]:
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [ ]:
users.info()

In [9]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

In [10]:
with open("restaurants.sql", "r") as f:
    sql_script = f.read()

cursor.executescript(sql_script)

In [11]:
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)

In [12]:
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [14]:
merged_df = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

In [15]:
final_df = pd.merge(
    merged_df,
    restaurants,
    on="restaurant_id",
    how="left"
)

In [16]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [ ]:
final_df.info()

In [ ]:
final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)


In [ ]:
final_df.groupby('cuisine')['total_amount'] \
    .mean() \
    .sort_values(ascending=False)

In [ ]:
user_spending = final_df.groupby('user_id')['total_amount'].sum()

(user_spending > 1000).sum()

In [ ]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels)


In [ ]:
final_df.groupby('rating_range')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)


In [ ]:
final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .mean() \
    .sort_values(ascending=False)

In [ ]:
restaurant_count = final_df.groupby('cuisine')['restaurant_id'] \
    .nunique() \
    .reset_index(name='restaurants')

revenue = final_df.groupby('cuisine')['total_amount'] \
    .sum() \
    .reset_index(name='revenue')

cuisine_analysis = pd.merge(
    restaurant_count,
    revenue,
    on='cuisine'
)

cuisine_analysis.sort_values(
    by=['restaurants', 'revenue'],
    ascending=[True, False]
)

In [ ]:
gold_orders = final_df[final_df['membership'] == 'Gold'].shape[0]
total_orders = final_df.shape[0]

round((gold_orders / total_orders) * 100)

In [ ]:

restaurant_stats = (
    final_df
    .groupby('restaurant_name_x', as_index=False)
    .agg(
        avg_order_value=('total_amount', 'mean'),
        order_count=('order_id', 'count')
    )
)

less than 20 orders
filtered = restaurant_stats.query("order_count < 20")


filtered.sort_values(
    by='avg_order_value',
    ascending=False
).head(1)

In [ ]:
final_df[['restaurant_name_x', 'restaurant_name_y']].drop_duplicates().head(10)

In [ ]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

filtered[filtered['restaurant_name_x'].isin(options)] \
    .sort_values('avg_order_value', ascending=False)

In [ ]:
# Step 1: Define only the MCQ options
valid_combinations = [
    ('Gold', 'Indian'),
    ('Gold', 'Italian'),
    ('Regular', 'Indian'),
    ('Regular', 'Chinese')
]

# Step 2: Filter to those combinations only
filtered_combos = (
    final_df
    .groupby(['membership', 'cuisine'], as_index=False)
    .agg(total_revenue=('total_amount', 'sum'))
)

filtered_combos = filtered_combos[
    filtered_combos[['membership', 'cuisine']]
    .apply(tuple, axis=1)
    .isin(valid_combinations)
]

# Step 3: Sort to find highest revenue among options
filtered_combos.sort_values('total_revenue', ascending=False)


In [ ]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'])

In [ ]:
final_df['quarter'] = final_df['order_date'].dt.quarter

In [ ]:
quarter_revenue = (
    final_df
    .groupby('quarter', as_index=False)
    .agg(total_revenue=('total_amount', 'sum'))
    .sort_values('total_revenue', ascending=False)
)

quarter_revenue

In [ ]:
gold_orders_count = final_df[final_df['membership'] == 'Gold'].shape[0]
gold_orders_count

In [ ]:
hyderabad_revenue = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()

round(hyderabad_revenue)

In [ ]:
distinct_users = final_df['user_id'].nunique()
distinct_users


In [ ]:
gold_avg_order_value = final_df[final_df['membership'] == 'Gold']['total_amount'].mean()

round(gold_avg_order_value, 2)

In [ ]:
high_rating_orders = final_df[final_df['rating'] >= 4.5].shape[0]
high_rating_orders


In [ ]:
gold_city_revenue = (
    final_df[final_df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue

In [ ]:
top_gold_city = gold_city_revenue.index[0]

gold_orders_top_city = final_df[
    (final_df['membership'] == 'Gold') &
    (final_df['city'] == top_gold_city)
].shape[0]

gold_orders_top_city